In [270]:
# Imports
import numpy as np #Mathematical tools
import matplotlib.pyplot as plt #Plots charts
import pandas as pd #Import and manage data sets
from sklearn.neural_network import MLPClassifier 
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
# Preprocessing library
from sklearn.preprocessing import Imputer
# Categorical data encoder library
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
# Splitting the data set into training and testing sets library
from sklearn.model_selection import train_test_split
#Feature scaling library
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix

In [271]:
# Imports part 2

from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

In [272]:
# Functions part 1

def best_output(row):
    # se o downloadTime é diferente de 100 para T2 e T3, ambos os targets completam o download
    if ((row.downloadTimeT2!=100)&(row.downloadTimeT3!=100)):
        # o melhor output será aquele para o qual o downloadTime é menor
        if (row.downloadTimeT2<=row.downloadTimeT3):
            return 0
        else:
            return 1
    
    # se o downloadTime é diferente de 100 apenas para um dos targets, só um deles completa o download
    elif ((row.downloadTimeT2!=100)|(row.downloadTimeT3!=100)):
        # o melhor output será aquele para o qual o downloadTime é diferente de 100
        if (row.downloadTimeT2!=100):
            return 0
        else:
            return 1
        
    # se o downloadTime é igual a 100 para T2 e T3, ambos os targets não completam o download
    elif ((row.downloadTimeT2==100)&(row.downloadTimeT3==100)):
        # o melhor output será aquele para o qual o rxBytes é maior
        if (row.rxBytesT2>=row.rxBytesT3):
            return 0
        else:
            return 1
        
def download_complete(row, column):
    if (row[column]==0):
        if(row.downloadTimeT2!=100.0):
            return 1
        else:
            return 0
    else:
        if(row.downloadTimeT3!=100.0):
            return 1
        else: 
            return 0

        
def download_time(row, column):
    if ((row[column]==0)&(row.downloadTimeT2<100)):
        return row.downloadTimeT2
    elif ((row[column]==1)&(row.downloadTimeT3<100)):
        return row.downloadTimeT3

    
def throughput(row, column):
    if (row[column]==0):
        tp = (row.rxBytesT2/row.downloadTimeT2)*8/1e6
        return tp
    else:
        tp = (row.rxBytesT3/row.downloadTimeT3)*8/1e6
        return tp
    
def optimum_choice(row, column):
    if (row[column]==row.best_output):
        return 1
    else:
        return 0
    
def a2a4rsrp(row):
    if (row.rsrp2>=row.rsrp3):
        return 0
    else:
        return 1

In [273]:
# Functions part 2
def percentageOfErrors(y_real, y_predicted):
    p= ((y_predicted != y_real).sum()/ y_real.size)*100
    return p 

def gettingEstimatorConfiguration(estimator, param_test,x_train, y_train, score):
    conf=GridSearchCV(estimator=estimator, param_grid=param_test, scoring=score, cv=10, n_jobs=-1)
    conf.fit(x_train, y_train)
    print(conf.best_estimator_)
    return conf.best_estimator_

def gettingTheConfigurationOfAllEstimatorsOfAType(estimators, param_test, train):
    x=np.array(train[0])
    y=np.array(train[1:3])
    # Level 2A
    est10=gettingEstimatorConfiguration(estimators[1,0], param_test,x[1], y[0,1],'neg_mean_squared_error')
    est11=gettingEstimatorConfiguration(estimators[1,1], param_test,x[1], y[1,1],'neg_mean_squared_error')    
    # Level 2B
    est20=gettingEstimatorConfiguration(estimators[2,0], param_test,x[2], y[0,2],'neg_mean_squared_error')
    est21=gettingEstimatorConfiguration(estimators[2,1], param_test,x[2], y[1,2],'neg_mean_squared_error')
    # Level 1
    est00=gettingEstimatorConfiguration(estimators[0,0], param_test,x[0], y[0,0],'accuracy')
    est01=gettingEstimatorConfiguration(estimators[0,1], param_test,x[0], y[1,0], 'accuracy')

    return [[est00,est01],[est10,est11],[est20, est21]]

def gettingAllEstimatorsConfigurations(train, randomState):
    # All test parameters
    param_test_knn = [{'n_neighbors':[(i) for i in range(5,31)], 'p':[1,2],'weights':['uniform', 'distance'],
                  'algorithm':['auto', 'brute']},{'n_neighbors':[(i) for i in range(5,31)], 'p':[1,2],
                 'weights':['uniform', 'distance'],
                  'algorithm':['ball_tree', 'kd_tree'], 'leaf_size':[10,20,30,40,50,60,70, 80,90,100]} ]
    param_test_svm =[{'C':[1,2,3,4,5,6,7,8,9,10,100,200, 300, 400, 500, 600, 700, 800, 900, 1000],'degree':[(i) for i in range(1,21)],'kernel':['poly'],'decision_function_shape':['ovo','ovr'],
                  'shrinking':[True, False]},
                 {'C':[1,2,3,4,5,6,7,8,9,10,100,200, 300, 400, 500, 600, 700, 800, 900, 1000],'kernel':['linear'],'decision_function_shape':['ovo','ovr'],
                  'shrinking':[True, False]},
                  {'C':[1,2,3,4,5,6,7,8,9,10,100,200, 300, 400, 500, 600, 700, 800, 900, 1000],'kernel':['rbf', 'sigmoid'],'gamma':[1,0.5,0.1, 0.01,0.001, 0.0001, 0.0000001],
                     'shrinking':[True, False]}] 
    # Creating the DT estimator parameters
    param_test_dt=[{ 'splitter':['best', 'random'], 'max_features':[None,'auto', 'sqrt', 'log2']}]    
   
    # Creating the RF estimator parameters
    param_test_rf = {'n_estimators':[(i) for i in range(5,31)],  'max_features':[None,'auto', 'sqrt', 'log2'],  
                     'verbose':[0], 'n_jobs':[-1], 'bootstrap':[True, False]}
    
    hl=[[12, 6, 3], [24, 12, 6],[10, 10, 10],[20,20,20],[30,30,30], [60], [120]]
   
    #param_test_nn = [{'activation' : ['identity', 'logistic', 'tanh', 'relu'], 'solver' :['lbfgs'],
     #           'learning_rate' : ['constant', 'invscaling', 'adaptive'], 'hidden_layer_sizes': hl,
     #                'max_iter':[500]},{'activation' : ['identity', 'logistic', 'tanh', 'relu'], 'solver' :['sgd', 'adam'],
     #           'learning_rate' : ['constant', 'invscaling', 'adaptive'], 'hidden_layer_sizes': hl,
     #                'max_iter':[500], 'early_stopping':[True, False], 'validation_fraction':[0.1,0.15,0.2],'shuffle':[True, False],
    #                                   'learning_rate_init':[0.001,0.01, 0.1]}]
    param_test_nn = [{'activation' : ['identity', 'logistic', 'tanh', 'relu'], 'solver' :['lbfgs'],
               'learning_rate' : ['constant', 'invscaling', 'adaptive'], 'hidden_layer_sizes': hl,
                    'max_iter':[400]}]   
        
    # All estimators
    # Level 1
    estimator_knn00=KNeighborsClassifier(metric = 'minkowski')
    estimator_svm00=SVC(random_state=randomState)
    estimator_dt00=DecisionTreeClassifier(random_state=randomState)
    estimator_rf00=RandomForestClassifier(random_state=randomState)
    estimator_nn00=MLPClassifier(random_state=randomState)
    
    estimator_knn01=KNeighborsClassifier(metric = 'minkowski')
    estimator_svm01=SVC(random_state=randomState)
    estimator_dt01=DecisionTreeClassifier(random_state=randomState)
    estimator_rf01=RandomForestClassifier(random_state=randomState)
    estimator_nn01=MLPClassifier(random_state=randomState)
    
    # Level 2A
    estimator_knn10=KNeighborsRegressor(metric = 'minkowski')
    estimator_svm10=SVR()
    estimator_dt10=DecisionTreeRegressor(random_state=randomState)
    estimator_rf10=RandomForestRegressor(random_state=randomState)
    estimator_nn10=MLPRegressor(random_state=randomState)
    
    estimator_knn11=KNeighborsRegressor(metric = 'minkowski')
    estimator_svm11=SVR()
    estimator_dt11=DecisionTreeRegressor(random_state=randomState)
    estimator_rf11=RandomForestRegressor(random_state=randomState)
    estimator_nn11=MLPRegressor(random_state=randomState)
    
    # Level 2B  
    estimator_knn20=KNeighborsRegressor(metric = 'minkowski')
    estimator_svm20=SVR(0)
    estimator_dt20=DecisionTreeRegressor(random_state=randomState)
    estimator_rf20=RandomForestRegressor(random_state=randomState)
    estimator_nn20=MLPRegressor(random_state=randomState)
    
    estimator_knn21=KNeighborsRegressor(metric = 'minkowski')
    estimator_svm21=SVR(0)
    estimator_dt21=DecisionTreeRegressor(random_state=randomState)
    estimator_rf21=RandomForestRegressor(random_state=randomState)
    estimator_nn21=MLPRegressor(random_state=randomState)
    
    estimators_knn=np.array([[estimator_knn00,estimator_knn01],[estimator_knn10,estimator_knn11],
                    [estimator_knn20,estimator_knn21]])
    estimators_nn=np.array([[estimator_nn00,estimator_nn01],[estimator_nn10,estimator_nn11],
                   [estimator_nn20,estimator_nn21]])
    estimators_svm=np.array([[estimator_svm00,estimator_svm01],[estimator_svm10,estimator_svm11],
                   [estimator_svm20,estimator_svm21]])
    estimators_dt=np.array([[estimator_dt00,estimator_dt01],[estimator_dt10,estimator_dt11],
                   [estimator_dt20,estimator_dt21]])
    estimators_rf=np.array([[estimator_rf00,estimator_rf01],[estimator_rf10,estimator_rf11],
                   [estimator_rf20,estimator_rf21]])
    
    #Finding the best configurations
    
    print("Best parameters for NN estimator")
    conf_nn=gettingTheConfigurationOfAllEstimatorsOfAType(estimators=estimators_nn, param_test=param_test_nn, 
                                                          train=train)   
    print("Best parameters for KNN estimator")
    conf_knn=gettingTheConfigurationOfAllEstimatorsOfAType(estimators=estimators_knn, param_test=param_test_knn, 
                                                          train=train)
    print("Best parameters for SVM estimator")
    conf_svm=gettingTheConfigurationOfAllEstimatorsOfAType(estimators=estimators_svm, param_test=param_test_svm, 
                                                          train=train)
    print("Best parameters for DT estimator")
    conf_dt=gettingTheConfigurationOfAllEstimatorsOfAType(estimators=estimators_dt, param_test=param_test_dt, 
                                                          train=train)
    print("Best parameters for RF estimator")
    conf_rf=gettingTheConfigurationOfAllEstimatorsOfAType(estimators=estimators_rf, param_test=param_test_rf, 
                                                          train=train)
    
    
    conf=[conf_knn,conf_svm, conf_dt, conf_rf, conf_nn]
    return conf

def theBestScore(oldBest, candidate):
    if oldBest>candidate:
        return oldBest
    else:
        return candidate    
def theWorstScore(oldWorst, candidate):
    if oldWorst<candidate:
        return oldWorst
    else:
        return candidate
    
def addArray(array1, array2):
    answer=[a1 + a2 for a1, a2 in zip(array1, array2)]
    return answer
    
    
def applyingAllEstimators (estimators,numberOfRandomStates,data):   
    # Initializing the variables
    # Averages
    p_svm=0
    p_dt=0
    p_rf=0
    p_knn=0
    p_nn=0
    # Worst cases 
    w_knn=100
    w_svm=100
    w_rf=100
    w_dt=100
    w_nn=100
    # Best cases
    b_svm=0
    b_dt=0
    b_rf=0
    b_knn=0
    b_nn=0
    
    #Other stats
    stats_knn=np.array([0,0,0,0])
    stats_svm=np.array([0,0,0,0])
    stats_dt=np.array([0,0,0,0])
    stats_rf=np.array([0,0,0,0])
    stats_nn=np.array([0,0,0,0])
    
    for (i) in range(0,numberOfRandomStates):
        #r1=np.random.randint(1,10001)
        #r2=np.random.randint(1,10001)
        r1=i
        r2=i
        # Splitting test data and train data
        [train, test, indexes, y_test]=splitingData(randomState=i, data=data) 
        print("{}: RandomState r1={} and r2={}".format(i, r1,r2))
        # Fitting the KNN to the training set
        stats=applyingEstimator(indexes=indexes,estimators=estimators[0],train=train, test=test, y_test=y_test)
        p=stats[3]
        stats_knn=addArray(stats_knn, stats)
        p_knn=p_knn+p
        w_knn=theWorstScore(oldWorst=w_knn, candidate=p)
        b_knn=theBestScore(oldBest=b_knn, candidate=p)
        # Fitting the SVM to the training set
        estimators[1]=changingRandomState(estimators=estimators[1], randomState=r2)
        stats=applyingEstimator(indexes=indexes,estimators=estimators[1],train=train, test=test, y_test=y_test)
        p=stats[3]
        stats_svm=addArray(stats_svm, stats)
        p_svm=p_svm+p
        w_svm=theWorstScore(oldWorst=w_svm, candidate=p)
        b_svm=theBestScore(oldBest=b_svm, candidate=p)
        # Fitting the DT to the training set
        estimators[2]=changingRandomState(estimators=estimators[2], randomState=r2)
        stats=applyingEstimator(indexes=indexes,estimators=estimators[2],train=train, test=test, y_test=y_test)
        p=stats[3]
        stats_dt=addArray(stats_dt, stats)
        p_dt=p_dt+p
        w_dt=theWorstScore(oldWorst=w_dt, candidate=p)
        b_dt=theBestScore(oldBest=b_dt, candidate=p)
        # Fitting the RF to the training set
        estimators[3]=changingRandomState(estimators=estimators[3], randomState=r2)
        stats=applyingEstimator(indexes=indexes,estimators=estimators[3],train=train, test=test, y_test=y_test)
        p=stats[3]
        stats_rf=addArray(stats_rf, stats)
        p_rf=p_rf+p
        w_rf=theWorstScore(oldWorst=w_rf, candidate=p)
        b_rf=theBestScore(oldBest=b_rf, candidate=p)
        # Fitting the NN to the training set
        estimators[4]=changingRandomState(estimators=estimators[4], randomState=r2)
        stats=applyingEstimator(indexes=indexes,estimators=estimators[4],train=train, test=test, y_test=y_test)
        p=stats[3]
        stats_nn=addArray(stats_nn, stats)
        p_nn=p_nn+p
        w_nn=theWorstScore(oldWorst=w_nn, candidate=p)
        b_nn=theBestScore(oldBest=b_nn, candidate=p)
        
    stats_knn=[elem / numberOfRandomStates for elem in stats_knn]
    stats_svm=[elem / numberOfRandomStates for elem in stats_svm]
    stats_dt=[elem / numberOfRandomStates for elem in stats_dt]
    stats_rf=[elem / numberOfRandomStates for elem in stats_rf]
    stats_nn=[elem / numberOfRandomStates for elem in stats_nn]
    
    stats_knn.extend([b_knn, w_knn])
    stats_svm.extend([b_svm, w_svm])
    stats_dt.extend([b_dt, w_dt])
    stats_rf.extend([b_rf, w_rf])
    stats_nn.extend([b_nn, w_nn])
    stats=np.array([stats_knn,stats_svm, stats_dt, stats_rf, stats_nn])
    
    tp=throughput(rxBytes=stats[:,1], downloadTime=stats[:,0])
    s=pd.DataFrame({'Algorithm':['KNN', 'SVM', 'DT', 'RF', 'NN'],'Average Score':stats[:,3], 'Best score': stats[:,4], 'Worst score': stats[:,5],'Downloads Completed':stats[:,2],'Download Time':stats[:,0], 'Received Bytes': stats[:,1], 'Throughput': tp})
    return s   
    

In [274]:
# Functions part 3
def calculatingStatistics(indexes, choice):
    downloadTime=[]
    rxBytes=[]
    downloadsCompleted=0
    for i in range(0, 200):
        index=indexes[i]
        if(choice[i]==0):
            downloadTime.extend([data.downloadTimeT2[index]])
            rxBytes.extend([data.rxBytesT2[index]])
        else:
            downloadTime.extend([data.downloadTimeT3[index]])
            rxBytes.extend([data.rxBytesT3[index]])
        if(downloadTime[i]!=100):
            downloadsCompleted=downloadsCompleted+1
    meanDownloadTime=np.mean(downloadTime) 
    meanRxBytes=np.mean(rxBytes)
    downloadsCompleted=downloadsCompleted/2 # Divided by 200 and multiplied by 100
    return[meanDownloadTime, meanRxBytes, downloadsCompleted]

float_formatter = lambda num: "%.2f" % num
np.set_printoptions(formatter={'float_kind':float_formatter})
pd.options.display.float_format = float_formatter

def throughput(rxBytes, downloadTime):
    tp=[i/j for i,j in zip(rxBytes, downloadTime)]
    return tp

def featureScaling(x_train,x_test):
    # Feature scaling (normalizing or stadardization of the scales)
    # Helps the conversion of the algorithm
    sc_X = StandardScaler()
    x_train = sc_X.fit_transform(x_train)
    x_test = sc_X.transform(x_test) # There is no need to fit after the training set is fit
    return[x_train, x_test]

def splitingData(data, randomState):
    # Splitting the data between independent variables and dependent variable
    y = data['best_output']
    x = data[['rsrp1','downloadTimeT2','downloadTimeT3', 'rxBytesT2','rxBytesT3','rsrq1','rsrp2','rsrq2','rsrp3','rsrq3','previousrsrp1','previousrsrq1','previousrsrp2','previousrsrq2','previousrsrp3','previousrsrq3']]

    # Splitting test data and train data
    x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = 0.20, random_state = randomState)
    indexes=x_train.axes[0]
    indexes=indexes.values

    data_train=data.loc[indexes]
    data_test=pd.concat([data, data_train]).drop_duplicates(keep=False)
    
    indexes=x_test.axes[0]

    # Level 1
    x1 = data_train[['rsrp1','rsrq1','rsrp2','rsrq2','rsrp3','rsrq3','previousrsrp1',
                     'previousrsrq1','previousrsrp2','previousrsrq2','previousrsrp3','previousrsrq3']]
    y1_t2 = (data_train.downloadTimeT2<100).astype(int).values
    y1_t3 = (data_train.downloadTimeT3<100).astype(int).values
    x1_test=data_test[['rsrp1','rsrq1','rsrp2','rsrq2','rsrp3','rsrq3','previousrsrp1',
                       'previousrsrq1','previousrsrp2','previousrsrq2','previousrsrp3','previousrsrq3']]
    y1_t2_test = (data_test.downloadTimeT2<100).astype(int).values
    y1_t3_test = (data_test.downloadTimeT3<100).astype(int).values
    [x1, x1_test]=featureScaling(x1,x1_test)  

    # Level 2A
    # Gets only the data of the simulations when both eNBs are able to finish the download
    x2 = data_train[['rsrp1','rsrq1','rsrp2','rsrq2','rsrp3','rsrq3','previousrsrp1','previousrsrq1',
                     'previousrsrp2','previousrsrq2','previousrsrp3','previousrsrq3']]
    y2_t2 = data_train.downloadTimeT2.values
    y2_t3= data_train.downloadTimeT3.values
    x2_test = data_test[['rsrp1','rsrq1','rsrp2','rsrq2','rsrp3','rsrq3','previousrsrp1',
                         'previousrsrq1','previousrsrp2','previousrsrq2','previousrsrp3','previousrsrq3']]
    y2_t2_test = data_test.downloadTimeT2.values
    y2_t3_test= data_test.downloadTimeT3.values
    [x2, x2_test]=featureScaling(x2,x2_test) 

    # Level 2B
    # Gets only the data of the simulations none of the eNBs are able to finish the download
    x3 = data_train[['rsrp1','rsrq1','rsrp2','rsrq2','rsrp3','rsrq3','previousrsrp1','previousrsrq1',
                     'previousrsrp2','previousrsrq2','previousrsrp3','previousrsrq3']]
    y3_t2 = data_train.rxBytesT2.values
    y3_t3= data_train.rxBytesT3.values
    x3_test = data_test[['rsrp1','rsrq1','rsrp2','rsrq2','rsrp3','rsrq3','previousrsrp1','previousrsrq1',
                         'previousrsrp2','previousrsrq2','previousrsrp3','previousrsrq3']]
    y3_t2_test = data_test.rxBytesT2.values
    y3_t3_test= data_test.rxBytesT3.values
    [x3, x3_test]=featureScaling(x3,x3_test) 

    train=[[x1, x2, x3], [y1_t2, y2_t2, y3_t2],[y1_t3, y2_t3, y3_t3]]
    test= [[x1_test, x2_test, x3_test], [y1_t2_test, y2_t2_test, y3_t2_test],[y1_t3_test, y2_t3_test, y3_t3_test]]

    
    return [train, test, indexes, y_test] 


In [275]:
# Functions part 4
# Estrategy 1 function

def predictOutput(x, estimators):
    # se o downloadTime previsto é menor que 100 para T2 e T3, ambos os targets completam o download
    pred=np.array([[estimators[0,0].predict(x), estimators[0,1].predict(x)], [estimators[1,0].predict(x),estimators[1,1].predict(x)],
                  [estimators[2,0].predict(x), estimators[2,1].predict(x)]])
    out=[]
    p00=pred[0,0]
    p01=pred[0,1]
    p10=pred[1,0]
    p11=pred[1,1]
    p20=pred[2,0]
    p21=pred[2,1]
    for (i) in range(0, 200):
        if ((p00[i]==1)&(p01[i]==1)):
            # Nível 2B
            # o melhor output será aquele para o qual o downloadTime previsto é menor
            if (p10[i]<=p11[i]):
                out.extend([0])
            else:
                out.extend([1])

        # se o downloadTime previsto é menor que 100 apenas para um dos targets, só um deles completa o download
        elif ((p00[i]==1)|(p01[i]==1)):
            # o melhor output será aquele para o qual o downloadTime previsto é menor que 100
            if (p00[i]==1):
                out.extend([0])
            else:
                out.extend([1])

        # se o downloadTime previsto é igual a 100 para ambos os targets
        elif ((p00[i]==0)&(p01[i]==0)):
            # o melhor output será aquele para o qual o rxBytes previsto é maior
            if (p20[i]>=p21[i]):
                out.extend([0]) 
            else:
                out.extend([1])
    return out

def applyingEstimator(estimators, train, test, indexes, y_test):
    x=np.array(train[0])
    y=np.array(train[1:3])
    x_test=np.array(test[0])
    #Fitting the estimators
    ## Level 1
    estimators[0,0].fit(x[0],y[0,0])
    estimators[0,1].fit(x[0],y[1,0])
    ## Level 2A
    estimators[1,0].fit(x[1],y[0,1])
    estimators[1,1].fit(x[1],y[1,1])
    ## Level 2B
    estimators[2,0].fit(x[2],y[0,2])
    estimators[2,1].fit(x[2],y[1,2])
    
    y_pred=predictOutput(x=x_test[0], estimators=estimators)
    stats=calculatingStatistics(indexes=indexes, choice=y_pred)
    
    print(out)
    print(y_test)
    
    # Evaluation of the model
    # Making the confusion matrix
    #cm = confusion_matrix(y_test, y_pred)
    p=percentageOfErrors(y_real=y_test, y_predicted=y_pred)
    s=np.array([ stats[0], stats[1], stats[2], 100 - p])
    print(s)
    return s

def changingRandomState(estimators, randomState):
    estimators[0,0].random_state=randomState
    estimators[0,1].random_state=randomState
    estimators[1,0].random_state=randomState
    estimators[1,1].random_state=randomState
    estimators[2,0].random_state=randomState
    estimators[2,1].random_state=randomState
    return estimators

# Sem shadowing

In [276]:
# Importing data

# importando os datasets
t2 = pd.read_csv('resultados/t2_OkumuraHata_Modificado', delimiter='\t')
t3 = pd.read_csv('resultados/t3_OkumuraHata_Modificado', delimiter='\t')

# garantindo que utilizaremos apenas as sementes presentes nos dois datasets
t2 = t2[t2.nRun.isin(t3.nRun)]
t3 = t3[t3.nRun.isin(t2.nRun)]
t2 = t2.reset_index(drop=True)
t3 = t3.reset_index(drop=True)

# combinando os datasets
data = t2
data = data.drop(['targetCellId', 'downloadTime', 'rxBytes'], axis=1)
data['downloadTimeT2'] = t2.downloadTime
data['downloadTimeT3'] = t3.downloadTime
data['rxBytesT2'] = t2.rxBytes
data['rxBytesT3'] = t3.rxBytes

data['best_output'] = data.apply(best_output, axis=1)
data.head()

[train,test, indexes, y_test]=splitingData(data=data, randomState=0)


In [ ]:
classifier_knn=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=6, p=1,
           weights='uniform')

classifier_svm=SVC(C=100, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovo', degree=3, gamma=1, kernel='rbf',
  max_iter=-1, probability=False, random_state=0, shrinking=True,
  tol=0.001, verbose=False)

classifier_dt=DecisionTreeClassifier(class_weight='balanced', criterion='gini',
            max_depth=None, max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=0,
            splitter='best')


classifier_rf=RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=None, max_features=None,
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=5, n_jobs=-1, oob_score=False, random_state=0,
            verbose=0, warm_start=False)

classifier_nn=MLPClassifier(activation='logistic', alpha=0.0001, batch_size='auto',
       beta_1=0.9, beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=[10,10,10], learning_rate='constant',
       learning_rate_init=0.001, max_iter=2000, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=0, shuffle=True,
       solver='lbfgs', tol=0.0001, validation_fraction=0.1, verbose=False,
       warm_start=False)

conf=[classifier_knn,classifier_svm,classifier_dt,classifier_rf, classifier_nn]

In [ ]:
# Comment this line if you want to use the estimators above
conf=gettingAllEstimatorsConfigurations(train=train, randomState=0)

# Setting the estimators
estimator_knn=conf[0]
estimator_svm=conf[1]
estimator_dt=conf[2]
estimator_rf=conf[3]
estimator_nn=conf[4]


estimators=np.array([estimator_knn,estimator_svm,estimator_dt,estimator_rf, estimator_nn])

Best parameters for NN estimator


In [ ]:
# Applying the classifiers
maxRange=100
stats = applyingAllEstimators(data=data,estimators=estimators,numberOfRandomStates=maxRange)

In [ ]:
print('Results without shadowing:')
print(stats)
stats

# Com shadowing

In [ ]:
# Importing data

# importando os datasets
t2 = pd.read_csv('resultados/t2_OhBuildings_ComShadowing_Modificado', delimiter='\t')
t3 = pd.read_csv('resultados/t3_OhBuildings_ComShadowing_Modificado', delimiter='\t')

# garantindo que utilizaremos apenas as sementes presentes nos dois datasets
t2 = t2[t2.nRun.isin(t3.nRun)]
t3 = t3[t3.nRun.isin(t2.nRun)]
t2 = t2.reset_index(drop=True)
t3 = t3.reset_index(drop=True)

# combinando os datasets
data = t2
data = data.drop(['targetCellId', 'downloadTime', 'rxBytes'], axis=1)
data['downloadTimeT2'] = t2.downloadTime
data['downloadTimeT3'] = t3.downloadTime
data['rxBytesT2'] = t2.rxBytes
data['rxBytesT3'] = t3.rxBytes

data['best_output'] = data.apply(best_output, axis=1)
data.head()

[train,test, indexes, y_test]=splitingData(data=data, randomState=0)

In [ ]:
classifier_nn=MLPClassifier(activation='tanh', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=[10, 10, 10], learning_rate='constant',
       learning_rate_init=0.001, max_iter=2000, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=0, shuffle=True,
       solver='lbfgs', tol=0.0001, validation_fraction=0.1, verbose=False,
       warm_start=False)

classifier_knn=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=21, p=2,
           weights='uniform')


classifier_svm=SVC(C=1000, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovo', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=0, shrinking=True,
  tol=0.001, verbose=False)

classifier_dt=DecisionTreeClassifier(class_weight='balanced', criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=0,
            splitter='best')

classifier_rf=RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='entropy', max_depth=None, max_features=None,
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=9, n_jobs=-1, oob_score=False, random_state=0,
            verbose=0, warm_start=False)

conf=[classifier_knn,classifier_svm,classifier_dt,classifier_rf, classifier_nn]


In [ ]:
# Comment this line if you want to use the estimators above
conf=gettingAllEstimatorsConfigurations(train=train, randomState=0)

# Setting the estimators
estimator_knn=conf[0]
estimator_svm=conf[1]
estimator_dt=conf[2]
estimator_rf=conf[3]
estimator_nn=conf[4]


estimators=np.array([estimator_knn,estimator_svm,estimator_dt,estimator_rf, estimator_nn])

In [ ]:
# Applying the classifiers
maxRange=200
stats = applyingAllClassifiers(numberOfRandomStates=maxRange, x=x, y=y, classifiers=classifiers)

In [ ]:
print('Results with shadowing:')
print(stats)
stats